In [13]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("../Nemotron_Personas.parquet").iloc[:3000,:]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   uuid                        3000 non-null   object
 1   professional_persona        3000 non-null   object
 2   sports_persona              3000 non-null   object
 3   arts_persona                3000 non-null   object
 4   travel_persona              3000 non-null   object
 5   culinary_persona            3000 non-null   object
 6   persona                     3000 non-null   object
 7   cultural_background         3000 non-null   object
 8   skills_and_expertise        3000 non-null   object
 9   skills_and_expertise_list   3000 non-null   object
 10  hobbies_and_interests       3000 non-null   object
 11  hobbies_and_interests_list  3000 non-null   object
 12  career_goals_and_ambitions  3000 non-null   object
 13  sex                         3000 non-null   obje

In [15]:
for column in df.columns:
    value=df.loc[1,column]
    print(f"Column: {column} \n Value: \n{value}\n")

Column: uuid 
 Value: 
50f90a6f17de473f9ca15f00afdedf7a

Column: professional_persona 
 Value: 
Alicia Gonzalez, a research scientist specializing in machine learning and natural language processing, channels her vivid imagination and competitive drive into designing generative algorithms that turn massive data sets into striking visual narratives, while her relaxed, sometimes disorganized work style lets her explore unconventional research paths, even if project timelines occasionally slip.

Column: sports_persona 
 Value: 
Alicia Gonzalez, an enthusiastic hiker who spends weekends trekking the Butte County trails with the Chico Outdoor Club, also supports the Sacramento Kings in basketball, follows the San Francisco 49ers in football, and plays in a local amateur soccer league during summer, while maintaining flexibility and balance through regular yoga sessions.

Column: arts_persona 
 Value: 
Alicia Gonzalez, an abstract painter and generative digital artist, draws creative fuel fr

In [16]:
df.columns

Index(['uuid', 'professional_persona', 'sports_persona', 'arts_persona',
       'travel_persona', 'culinary_persona', 'persona', 'cultural_background',
       'skills_and_expertise', 'skills_and_expertise_list',
       'hobbies_and_interests', 'hobbies_and_interests_list',
       'career_goals_and_ambitions', 'sex', 'age', 'marital_status',
       'education_level', 'bachelors_field', 'occupation', 'city', 'state',
       'zipcode', 'country'],
      dtype='object')

In [17]:
not_required_columns=['zipcode', 'country']
persona_columns=['persona', 'professional_persona', 'sports_persona',
                'arts_persona', 'travel_persona', 'culinary_persona',"persona"]
categorical_columns=['sex', 'marital_status','education_level', 
                     'bachelors_field', 'occupation', 'city', 'state']

In [18]:
df[persona_columns].head()

,persona,professional_persona,sports_persona,arts_persona,travel_persona,culinary_persona,persona
0,"Mary Alberti is a routine‑obsessed, bullet‑jou...",Mary Alberti is a front‑line food service spec...,Mary Alberti fuels their fitness routine by cl...,Mary Alberti finds creative inspiration in the...,Mary Alberti prefers meticulously planned week...,Mary Alberti showcases an intermediate culinar...,"Mary Alberti is a routine‑obsessed, bullet‑jou..."
1,Alicia Gonzalez fuses scientific precision wit...,"Alicia Gonzalez, a research scientist speciali...","Alicia Gonzalez, an enthusiastic hiker who spe...","Alicia Gonzalez, an abstract painter and gener...","Alicia Gonzalez, who mixes meticulous research...","Alicia Gonzalez, an intermediate‑to‑advanced h...",Alicia Gonzalez fuses scientific precision wit...
2,"Deeva Cintron, an 85‑year‑old garden‑loving qu...","Deeva Cintron, an 85‑year‑old community financ...","Deeva Cintron, a devoted fan of the Detroit Ti...","Deeva Cintron, an avid quilt maker and histori...","Deeva Cintron, who prefers meticulously planne...","Deeva Cintron, who delights in homestyle cooki...","Deeva Cintron, an 85‑year‑old garden‑loving qu..."
3,"Maria Buendia, a 34‑year‑old, channels their c...","Maria Buendia, a 34‑year‑old community‑focused...","Maria Buendia, a 34‑year‑old, stays active by ...","Maria Buendia, a 34‑year‑old, immerses themsel...","Maria Buendia, a 34‑year‑old, prefers weekend ...","Maria Buendia, a 34‑year‑old, blends heritage ...","Maria Buendia, a 34‑year‑old, channels their c..."
4,Julio Simmons is a 49‑year‑old restless polyma...,"Julio Simmons, a 49‑year‑old interdisciplinary...","Julio Simmons, an avid hiker of the Finger Lak...",Julio Simmons immerses themselves in a spectru...,Julio Simmons favours purposeful travel that f...,Julio Simmons approaches cooking as an experim...,Julio Simmons is a 49‑year‑old restless polyma...


In [19]:
df[categorical_columns].nunique()

sex                   2
marital_status        5
education_level       7
bachelors_field       6
occupation          349
city               1711
state                52
dtype: int64

## expertise_list

-TF-IDF 

In [20]:
import ast
df['hobbies_and_interests_list'] = df['hobbies_and_interests_list'].apply(ast.literal_eval)

In [21]:
total_hobbies = set()
for hobbies in df['hobbies_and_interests_list']:
    total_hobbies.update(hobbies)


len(total_hobbies)

12751

In [22]:
total_hobbies

{'Korean cooking',
 'sketching futuristic vehicle designs',
 'landscape photography',
 'Coastal Fishing',
 'Listening to banda, norteña, and ranchera music',
 'Volunteer work at community cultural events',
 'Volunteering at church and senior center',
 'Home woodworking projects',
 'DIY home crafts',
 'Playing in Neighborhood Softball Leagues',
 'collecting shells and pebbles at the beach',
 'Ultimate Frisbee',
 'Art Fairs',
 'Volunteering at senior reading program',
 'Weight training',
 'Volunteering at senior center (craft sessions)',
 'Lunar New Year community events',
 'Weekend road trips to historic towns',
 'Mentoring youth in STEM programs',
 'Solving crossword and Sudoku puzzles',
 'crayon drawing and coloring',
 'Participating in local history tours',
 'Attending public lectures on social innovation',
 'Reading popular science and historical non-fiction',
 'Classic country music listening',
 'Reading home-improvement and sustainability blogs',
 'Walking in Herrick Lake Forest P

In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

df_combined=pd.DataFrame()

# # Combine the text columns into a single 'full_persona' column
text_columns = persona_columns+['skills_and_expertise', 'skills_and_expertise_list', 'hobbies_and_interests', 'career_goals_and_ambitions']
df_combined['full_persona'] = df[text_columns].apply(lambda row: ' \n'.join(row.values.astype(str)), axis=1)


In [24]:
df_combined.head()

,full_persona
0,"Mary Alberti is a routine‑obsessed, bullet‑jou..."
1,Alicia Gonzalez fuses scientific precision wit...
2,"Deeva Cintron, an 85‑year‑old garden‑loving qu..."
3,"Maria Buendia, a 34‑year‑old, channels their c..."
4,Julio Simmons is a 49‑year‑old restless polyma...


In [13]:

# # Create a single vector for the combined text
model = SentenceTransformer('Qwen/Qwen3-Embedding-0.6B')
test_sentence = model.encode(df_combined['full_persona'].iloc[0:10].tolist())
# df_combined['persona_vector'] = df_combined['full_persona'].apply(lambda text: model.encode(text))


In [14]:
import pickle



PICKLE_FILE = '../persona_embeddings(Qwen.Qwen3-Embedding-0.6B).pkl'

try:
    # 1. Try to access the variable.
    # We use '_' to just check existence without doing anything.
    _ = test_sentence
    print("Variable 'test_sentence' already exists in memory.")

except NameError:
    # 2. If it raises a NameError, it doesn't exist.
    print("Variable not in memory. Trying to load from pickle...")
    
    try:
        # 3. Try to load it from the file.
        with open(PICKLE_FILE, 'rb') as f:
            test_sentence = pickle.load(f)
        print("Successfully loaded 'test_sentence' from file.")

    except FileNotFoundError:
        # 4. Handle the case where the file doesn't exist.
        with open(PICKLE_FILE, 'wb') as f:
            pickle.dump(test_sentence, f)
        print(f"Pickle file '{PICKLE_FILE}' has been created with 'test_sentence'.")
        
    except Exception as e:
        # 5. Handle other potential pickle errors (corrupt file, etc.)
        print(f"An error occurred while loading the pickle file: {e}")

# Now you can safely use the variable (if it was loaded or existed)
# You might want another check here if loading could fail
if 'test_sentence' in locals():
    print(f"\nFinal variable content: {test_sentence}")
else:
    print("\nVariable could not be loaded or created.")

Variable 'test_sentence' already exists in memory.

Final variable content: [[ 0.02852376 -0.00285406 -0.011209   ... -0.02703547 -0.03101355
  -0.02318499]
 [ 0.05625721 -0.02447899 -0.01034437 ... -0.00968731  0.02858953
  -0.03522873]
 [ 0.03395768 -0.01865099 -0.00975748 ...  0.01198935  0.01847553
  -0.00542701]
 ...
 [ 0.0174566   0.01496234 -0.00377005 ... -0.01245713 -0.02745259
  -0.04677221]
 [ 0.05269084  0.00078775 -0.00686584 ...  0.05289321 -0.03211106
  -0.02075862]
 [ 0.01816634  0.02290567 -0.00803386 ...  0.01226232  0.02619584
  -0.06010284]]


In [25]:
import json

JSON_FILE = '../all_hobbies.json'

try:
    with open(JSON_FILE, 'r') as f:
        total_hobbies = json.load(f)
    print("Successfully loaded 'total_hobbies' from JSON file.")

except FileNotFoundError:
    print(f"JSON file '{JSON_FILE}' not found.")
    with open(JSON_FILE, 'w') as f:
        json.dump(list(total_hobbies), f)

except json.JSONDecodeError:
    print(f"Error decoding JSON from file '{JSON_FILE}'.")


JSON file '../all_hobbies.json' not found.
